In [1]:
import os, sys
# add root working folder into path so that we can import abs paths in jupyter
# dir2 = os.path.abspath('')
# dir1 = os.path.dirname(dir2)
# if not dir1 in sys.path: sys.path.append(dir1)

In [2]:
%cd "D:\Schoolwork\TERM 3\WORK\visual_prosody"

D:\Schoolwork\TERM 3\WORK\visual_prosody


In [19]:
import argparse
import yaml
import numpy as np
from pprint import pprint

In [4]:
import argparse
import os

import torch
import yaml
import torch.nn as nn
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter
# from tqdm import tqdm

from utils.model import get_model, get_vocoder, get_param_num
from utils.tools import to_device, log, synth_one_sample
from model import FastSpeech2Loss
from dataset import Dataset
from utils.auto_tqdm import tqdm

from evaluate import evaluate

In [5]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [6]:
parser = argparse.ArgumentParser()
parser.add_argument("--restore_step", type=int, default=0)
parser.add_argument(
    "-p",
    "--preprocess_config",
    type=str,
    required=True,
    help="path to preprocess.yaml",
)
parser.add_argument(
    "-m", "--model_config", type=str, required=True, help="path to model.yaml"
)
parser.add_argument(
    "-t", "--train_config", type=str, required=True, help="path to train.yaml"
)

argString = '-p ./config/Ego4D_final_v3/preprocess.yaml -m ./config/Ego4D_final_v3/model.yaml -t ./config/Ego4D_final_v3/train.yaml'
# args = parser.parse_args()
args = parser.parse_args(argString.split())



In [7]:
# Read Config
preprocess_config = yaml.load(
    open(args.preprocess_config, "r"), Loader=yaml.FullLoader
)
model_config = yaml.load(open(args.model_config, "r"), Loader=yaml.FullLoader)
train_config = yaml.load(open(args.train_config, "r"), Loader=yaml.FullLoader)
configs = (preprocess_config, model_config, train_config)

In [8]:
# main()

In [9]:
print("Prepare training ...")

preprocess_config, model_config, train_config = configs

# Get dataset
dataset = Dataset(
    "train.txt", 'train', preprocess_config, train_config, sort=True, drop_last=True
)

Prepare training ...


In [10]:
len(dataset)

25951

In [13]:
pprint(dataset[0])

{'duration': array([ 2,  2,  2, 28,  2, 10,  4,  6, 53,  7]),
 'energy': array([0.9789963 , 0.95960546, 1.1227531 , 2.533922  , 4.8028474 ,
       4.474254  , 1.1891582 , 0.9398053 , 2.483526  , 1.7764769 ],
      dtype=float32),
 'id': '00025784-9ed5-4071-a84f-32e5e94e96a1',
 'mel': array([[ 2.9975373e-01, -1.6556007e-01, -6.5398759e-01, ...,
        -7.0371947e+00, -7.0798965e+00, -7.1081305e+00],
       [-3.5277855e-01, -7.1158201e-01, -5.5675012e-01, ...,
        -7.7303562e+00, -7.7730618e+00, -7.8012867e+00],
       [-1.7843202e-01, -1.1256552e+00, -5.0544268e-01, ...,
        -1.1512925e+01, -1.1512925e+01, -1.1512925e+01],
       ...,
       [ 2.7289128e-01, -8.7187129e-01,  8.3005680e-03, ...,
        -1.1512925e+01, -1.1512925e+01, -1.1512925e+01],
       [ 4.7123089e-01, -4.6125448e-01,  5.5685200e-02, ...,
        -1.1512925e+01, -1.1512925e+01, -1.1512925e+01],
       [ 2.4691910e-01, -9.1769451e-01, -9.6002296e-02, ...,
        -6.8397665e+00, -6.8557754e+00, -6.8668313e+

In [14]:
batch_size = train_config["optimizer"]["batch_size"]
group_size = 4  # Set this larger than 1 to enable sorting in Dataset
assert batch_size * group_size < len(dataset)
loader = DataLoader(
    dataset,
    batch_size=batch_size * group_size,
    shuffle=True,
    collate_fn=dataset.collate_fn,
)

In [16]:
for data in loader:
    break

In [25]:
len(data)

4

In [26]:
for item in data:
    if isinstance(item, np.ndarray):
        print(item.shape)
    else:
        print(len(item))

12
12
12
12


In [28]:
# Prepare model
model, optimizer = get_model(args, configs, device, train=True)
model = nn.DataParallel(model)
num_param = get_param_num(model)
Loss = FastSpeech2Loss(preprocess_config, model_config).to(device)
print("Number of FastSpeech2 Parameters:", num_param)

# Load vocoder
vocoder = get_vocoder(model_config, device)

Number of FastSpeech2 Parameters: 35159361
Removing weight norm...


In [29]:
# Init logger
for p in train_config["path"].values():
    os.makedirs(p, exist_ok=True)
train_log_path = os.path.join(train_config["path"]["log_path"], "train")
val_log_path = os.path.join(train_config["path"]["log_path"], "val")
os.makedirs(train_log_path, exist_ok=True)
os.makedirs(val_log_path, exist_ok=True)
train_logger = SummaryWriter(train_log_path)
val_logger = SummaryWriter(val_log_path)